In [1]:
# language translation

from transformers import pipeline
from tqdm import tqdm

def translation(inputs_fr):
    model_checkpoint = "Helsinki-NLP/opus-mt-fr-en"
    translator = pipeline("translation", model=model_checkpoint)

    inputs = []
    for i_fr in tqdm(inputs_fr):
        inp = translator(i_fr)

        inp = inp[0]['translation_text']

        inputs.append(inp)
    
    return inputs

C:\Users\avdhi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# quantifying intimacy score

# Import generic wrappers
from transformers import AutoModelForSequenceClassification, AutoTokenizer 
import torch

def calculate_intimacy_score(inputs, label=None):
    # Define the model repo
    model_name = "pedropei/question-intimacy" 


    # Download pytorch model
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
    # list of strings should be pretokenized
    # add_prefix_space needed for pretokenized
    
    t = tokenizer(inputs, return_tensors="pt", is_split_into_words=True,
                 padding=True, truncation=True, max_length=50, add_special_tokens=True)
    if label:
        labels = torch.tensor(label).unsqueeze(0)
        # Model apply
        output = model(**t, labels=labels)
    
    else:
        output = model(**t)

    return output

In [ ]:
inputs_fr = ["Quelle est la question que vous détestez qu'on vous pose ?", 
             "Quelle est l'importance de l'aérodynamique dans l'espace ?", 
             "Quelle est la meilleure façon de leur faire avoir des analsecks ?", 
             "Qu'est-ce que l'amour à 11 ans ?"]

labels = [0.517, -0.421, 0.4999, 0.514]

inp_en = translation(inputs_fr)
print(inp_en)

score = calculate_intimacy_score(inp_en, labels)
print(score)

In [4]:
import import_ipynb
from load_data import load_data, getLanguageData, k_folds, k_fold_split, shuffle_data

k=10

raw_data = load_data("data", "train_copy.csv")['train']

#English
print('English')
en_text, en_label = getLanguageData(raw_data, 'English')

#French
print('\nFrench')
fr_text, fr_label = getLanguageData(raw_data, 'French')
fr_text_folds,fr_label_folds = k_folds(k, [fr_text], [fr_label])

#translation
text = en_text
label = en_label
print("Translating French to English")
for i in range(k):
    translated_text = translation(fr_text_folds[i])
    text.extend(translated_text)
    label.extend(fr_label_folds[i])
print('text:',len(text))

# label = en_label
# label.extend(fr_label)
print('Train_label:',len(label))

#Shuffle 
text, label = shuffle_data(text, label)

print("Predicting Intimacy Scores")
score = calculate_intimacy_score(text, label)
print(score)

importing Jupyter notebook from load_data.ipynb


Using custom data configuration data-05d203ae109cc829
Found cached dataset csv (C:/Users/avdhi/.cache/huggingface/datasets/csv/data-05d203ae109cc829/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 124.47it/s]


English
Folding...
Fold 1 : 793 data
Fold 2 : 794 data
Split 1 - Training Data: 793 - Validation Data: 793
Split 2 - Training Data: 794 - Validation Data: 794

French


Using custom data configuration data-05d203ae109cc829
Found cached dataset csv (C:/Users/avdhi/.cache/huggingface/datasets/csv/data-05d203ae109cc829/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Folding...
Fold 1 : 794 data
Fold 2 : 794 data
Split 1 - Training Data: 794 - Validation Data: 794
Split 2 - Training Data: 794 - Validation Data: 794


...

English



French
Folding...
Fold 1 : 158 data
Fold 2 : 158 data
Fold 3 : 158 data
Fold 4 : 158 data
Fold 5 : 158 data
Fold 6 : 158 data
Fold 7 : 158 data
Fold 8 : 158 data
Fold 9 : 158 data
Fold 10 : 166 data
Translating French to English


C:\Users\avdhi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
100%|████████████████████████████████████████████████████████████████████████████████| 166/166 [04:44<00:00,  1.72s/it]


text: 3175
Train_label: 3175


NameError: name 'np' is not defined

In [12]:
import numpy as np

#Shuffle 
# text, label = shuffle_data(text, label)

print("Predicting Intimacy Scores")
score = calculate_intimacy_score(text, label)
print(score)

Predicting Intimacy Scores


Some weights of the model checkpoint at pedropei/question-intimacy were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SequenceClassifierOutput(loss=tensor(0.1255, grad_fn=<MseLossBackward0>), logits=tensor([[-0.2155]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


C:\Users\avdhi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([3175])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [14]:
import import_ipynb
from load_data import load_data, getLanguageData, k_folds, k_fold_split, shuffle_data

#Shuffle 
text, label = shuffle_data(text, label)

print("Predicting Intimacy Scores")
score = calculate_intimacy_score(text, label)
print(score)

NameError: name 'np' is not defined